In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-01-11T15:47:50.757933-05:00

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 8.10.0

Compiler    : Clang 15.0.0 (clang-1500.0.40.1)
OS          : Darwin
Release     : 23.0.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: Install the gpu extra (pip install 'watermark[gpu]') to display GPU information for NVIDIA chipsets



In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import sklearn
import gc
import os
import xgboost

In [5]:
%watermark --iversions

xgboost: 2.0.3
numpy  : 1.26.3
sklearn: 1.3.2
pandas : 2.1.4



In [6]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [7]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [19]:
optuna_log = pd.read_csv('optuna_xgb_output_0.csv')
optuna_log.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.260580,0.518387,0.588129,logloss,0.803620,0.001024,19,73,binary:logistic,0.734509,gpu_hist,COMPLETE
1,1,0.272333,0.554771,0.386440,logloss,0.515043,0.004684,19,226,binary:logistic,0.740431,gpu_hist,COMPLETE
2,2,0.273310,0.001337,0.321878,logloss,0.224638,0.035482,25,235,binary:logistic,0.647692,gpu_hist,COMPLETE
3,3,0.213742,0.475032,0.907356,logloss,0.025882,0.019162,21,5,binary:logistic,0.939027,gpu_hist,COMPLETE
4,4,0.250605,7.983068,0.797204,logloss,1.452732,0.037789,8,12,binary:logistic,0.532211,gpu_hist,COMPLETE


In [20]:
optuna_log.value.max()

0.2834224878819652

In [22]:
max_value_row = optuna_log[optuna_log['value'] == optuna_log['value'].max()]

In [23]:
max_value_row

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
94,94,0.283422,0.654147,0.917583,logloss,4.645511,0.013001,7,194,binary:logistic,0.66007,gpu_hist,COMPLETE


In [24]:
columns = test.columns[1:]

In [25]:
X = train[columns].values
X_test = test[columns].values
Y = train.target.values

In [26]:
XGBClassifier()

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [27]:
%%time
train_oof = np.zeros((X.shape[0], ))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = XGBClassifier(n_estimators=1000, max_depth=7, random_state=42, subsample=0.66, colsample_bytree=0.917, 
                          learning_rate=0.013, min_child_weight=194, reg_lambda=4.645511, reg_alpha=0.654147, tree_method='hist')
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)[:,1]
    train_oof[val_index] = val_pred
    print("Fold normalized:", gini_normalized(val_target, val_pred))
    test_preds += model.predict_proba(X_test)[:,1]/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold normalized: 0.2869842759265644
Fitting fold 2
Fold normalized: 0.29256416011081404
Fitting fold 3
Fold normalized: 0.2743080499620639
Fitting fold 4
Fold normalized: 0.28425047613223925
Fitting fold 5
Fold normalized: 0.27975393328512105
CPU times: user 10min 5s, sys: 13min 29s, total: 23min 35s
Wall time: 1min 13s


In [28]:
gini_normalized(Y, train_oof)

0.2833716890469155

In [29]:
roc_auc_score(Y, train_oof)

0.6416858437197219

In [30]:
sample_submission['target'] = test_preds

In [31]:
sample_submission.to_csv('../submissions/XGB_5_fold_optuna_best.csv.zip', index=False, compression='zip')

On the leaderbaord this submission scores 0.28013 on the public test set, and 0.2845 on the private dataset.